In [1]:
import numpy as np
X = np.load('datasets/delivery_review.npy')
y = np.load('datasets/delivery_label.npy')

In [2]:
with open('datasets/chinese_stop_words.txt', encoding="utf-8") as text:
    stopwords = [line.strip() for line in  text]

In [3]:
import jieba
X_new = []
for i, review in enumerate(X):
    review = list(jieba.cut(review))
    result = []
    for word in review:
        if word not in stopwords:
            result.append(word)
    X_new.append(" ".join(result))
X_new = np.array(X_new)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BEIZHO~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.786 seconds.
Prefix dict has been built succesfully.


In [4]:
X_new[1]

'分量 特色 下回 不吃'

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, shuffle=True)

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

maxlen = max([len(sentence.split(" ")) for sentence in X_new])

words_size = 10000
tokenizer = Tokenizer(num_words=words_size)
tokenizer.fit_on_texts(X_new)

sequences = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(sequences, 
                        maxlen=maxlen, 
                        padding='pre')

sequences = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(sequences, 
                       maxlen=maxlen, 
                       padding='pre')

Using TensorFlow backend.


In [7]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense
from keras.optimizers import Adam

embedding_dim = 32
model = Sequential()
model.add(Embedding(input_dim=words_size, 
                    input_length=maxlen,
                    output_dim=embedding_dim))
model.add(GRU(units=256))
model.add(Dense(units=1, 
                activation='sigmoid'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 135, 32)           320000    
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               221952    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 542,209
Trainable params: 542,209
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train,
          y_train,
          epochs=5,
          batch_size=32,
          verbose=2,
          validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Train on 5120 samples, validate on 1280 samples
Epoch 1/5
 - 52s - loss: 0.4999 - acc: 0.7639 - val_loss: 0.3806 - val_acc: 0.8336
Epoch 2/5
 - 46s - loss: 0.2929 - acc: 0.8855 - val_loss: 0.3590 - val_acc: 0.8547
Epoch 3/5
 - 47s - loss: 0.2308 - acc: 0.9133 - val_loss: 0.3949 - val_acc: 0.8445
Epoch 4/5
 - 47s - loss: 0.1744 - acc: 0.9354 - val_loss: 0.4315 - val_acc: 0.8297
Epoch 5/5
 - 50s - loss: 0.1399 - acc: 0.9463 - val_loss: 0.5090 - val_acc: 0.8305
